# Análise e particionamento dos dados iwildcam2020

#### Bibliotecas necessárias

In [848]:
import math

import pandas as pd
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import seaborn as sns

%matplotlib inline

### Carregando datasets iwildcam2020

In [849]:
images_train_path = '/data/alberto/iWildCam2020/data/train/'


In [850]:
file = '/data/alberto/iWildCam2020/data/metadata/iwildcam2020_train_annotations.json'
with open(file) as json_data:
    data = json.load(json_data)
annotations = pd.DataFrame.from_dict(data['annotations']).set_index('id')
images = pd.DataFrame.from_dict(data['images']).set_index('file_name')[['seq_id', 'frame_num', 'seq_num_frames', 'datetime', 'location', 'height', 'width']]
classes = pd.DataFrame.from_dict(data['categories']).set_index('id')

In [851]:
test_file = '/data/alberto/iWildCam2020/data/metadata/iwildcam2020_test_information.json'
with open(test_file) as json_data:
    test_data = json.load(json_data)
test_images = pd.DataFrame.from_dict(test_data['images']).set_index('file_name')[['seq_id', 'frame_num', 'seq_num_frames', 'datetime', 'location', 'height', 'width']]
test_classes = pd.DataFrame.from_dict(test_data['categories']).set_index('id')

### Fazendo a junção das instancias

In [852]:
instances = annotations.copy()
instances['image_id'] = instances['image_id'] + '.jpg'
instances = instances.join(images, on='image_id')
instances = instances.join(classes[['name']], on='category_id')

In [853]:
instances.head()

,count,image_id,category_id,seq_id,frame_num,seq_num_frames,datetime,location,height,width,name
id,,,,,,,,,,,
a292dd3c-21bc-11ea-a13a-137349068a90,1,96b00332-21bc-11ea-a13a-137349068a90.jpg,73,96b004ea-21bc-11ea-a13a-137349068a90,1,1,2013-08-08 11:45:00.000,267,1222,1795,canis lupus
a0afcfc0-21bc-11ea-a13a-137349068a90,1,879d74d8-21bc-11ea-a13a-137349068a90.jpg,4,879d7654-21bc-11ea-a13a-137349068a90,1,1,2013-08-01 22:31:00.000,264,1772,2739,cuniculus paca
a306e9c0-21bc-11ea-a13a-137349068a90,1,9017f7aa-21bc-11ea-a13a-137349068a90.jpg,227,9017f8cc-21bc-11ea-a13a-137349068a90,1,1,2013-04-17 11:15:00.000,45,1222,1891,psophia crepitans
9eed94c4-21bc-11ea-a13a-137349068a90,1,90d93c58-21bc-11ea-a13a-137349068a90.jpg,250,90d93e38-21bc-11ea-a13a-137349068a90,1,1,2013-04-23 08:26:00.000,45,1222,1795,aguila sp
a2a4dd7a-21bc-11ea-a13a-137349068a90,1,887cd0ec-21bc-11ea-a13a-137349068a90.jpg,2,887cd29a-21bc-11ea-a13a-137349068a90,1,1,2013-04-24 13:51:00.000,45,1196,1812,tayassu pecari


### Particionando as instâncias com base nas categorias

#### Nessa etapa, é feita a filtragem das categorias para armazená-las em 3 pastas : 
    1 - pasta_no_empty : Nela conterá somente as categorias dos animais, todos os animais
    2 - pasta_empty : Nela conterá somente a própria categoria empty
    3 - pasta_descarte : Nela conterá 11 categorias descartadas no experimento. As categorias dessa pasta são:             'end','misfire', 'motorcycle', 'start','unidentifiable', 'unknown', 'unknown bat', 'unknown bird',                 'unknown dove', 'unknown raptor', 'unknown rat'

#### Criando a pasta contendo somente as categorias dos animais

In [854]:
pasta_no_empty = instances.loc[instances['name']!='empty']
pasta_no_empty = instances.loc[instances['name']!='end']
pasta_no_empty = instances.loc[instances['name']!='misfire']
pasta_no_empty = instances.loc[instances['name']!='motorcycle']
pasta_no_empty = instances.loc[instances['name']!='start']
pasta_no_empty = instances.loc[instances['name']!='unidentifiable']
pasta_no_empty = instances.loc[instances['name']!='unknown']
pasta_no_empty = instances.loc[instances['name']!='unknown bat']
pasta_no_empty = instances.loc[instances['name']!='unknown bird']
pasta_no_empty = instances.loc[instances['name']!='unknown dove']
pasta_no_empty = instances.loc[instances['name']!='unknown rat']

In [855]:
pasta_no_empty

,count,image_id,category_id,seq_id,frame_num,seq_num_frames,datetime,location,height,width,name
id,,,,,,,,,,,
a292dd3c-21bc-11ea-a13a-137349068a90,1,96b00332-21bc-11ea-a13a-137349068a90.jpg,73,96b004ea-21bc-11ea-a13a-137349068a90,1,1,2013-08-08 11:45:00.000,267,1222,1795,canis lupus
a0afcfc0-21bc-11ea-a13a-137349068a90,1,879d74d8-21bc-11ea-a13a-137349068a90.jpg,4,879d7654-21bc-11ea-a13a-137349068a90,1,1,2013-08-01 22:31:00.000,264,1772,2739,cuniculus paca
a306e9c0-21bc-11ea-a13a-137349068a90,1,9017f7aa-21bc-11ea-a13a-137349068a90.jpg,227,9017f8cc-21bc-11ea-a13a-137349068a90,1,1,2013-04-17 11:15:00.000,45,1222,1891,psophia crepitans
9eed94c4-21bc-11ea-a13a-137349068a90,1,90d93c58-21bc-11ea-a13a-137349068a90.jpg,250,90d93e38-21bc-11ea-a13a-137349068a90,1,1,2013-04-23 08:26:00.000,45,1222,1795,aguila sp
a2a4dd7a-21bc-11ea-a13a-137349068a90,1,887cd0ec-21bc-11ea-a13a-137349068a90.jpg,2,887cd29a-21bc-11ea-a13a-137349068a90,1,1,2013-04-24 13:51:00.000,45,1196,1812,tayassu pecari
...,...,...,...,...,...,...,...,...,...,...,...
9b4c9de2-21bc-11ea-a13a-137349068a90,1,8b7899a2-21bc-11ea-a13a-137349068a90.jpg,372,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-08-01 11:39:03.000,218,1024,1280,meleagris ocellata
9a379740-21bc-11ea-a13a-137349068a90,1,958f609c-21bc-11ea-a13a-137349068a90.jpg,372,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-08-01 11:39:04.000,218,1024,1280,meleagris ocellata
9d0a3c0c-21bc-11ea-a13a-137349068a90,1,93bd2ede-21bc-11ea-a13a-137349068a90.jpg,372,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-08-01 11:39:05.000,218,1024,1280,meleagris ocellata


#### Criando a pasta contendo a categoria empty, ou seja, somente o background

In [856]:
pasta_empty = instances.loc[instances['name']=='empty']

In [857]:
pasta_empty

,count,image_id,category_id,seq_id,frame_num,seq_num_frames,datetime,location,height,width,name
id,,,,,,,,,,,
9ab32f72-21bc-11ea-a13a-137349068a90,-1,888e07e0-21bc-11ea-a13a-137349068a90.jpg,0,8ede4f1a-21bc-11ea-a13a-137349068a90,1,3,2013-02-05 10:12:37.000,125,1024,1280,empty
9aafd386-21bc-11ea-a13a-137349068a90,-1,8ede4d44-21bc-11ea-a13a-137349068a90.jpg,0,8ede4f1a-21bc-11ea-a13a-137349068a90,2,3,2013-02-05 10:12:38.000,125,1024,1280,empty
9d80ea64-21bc-11ea-a13a-137349068a90,-1,868b2ab8-21bc-11ea-a13a-137349068a90.jpg,0,8ede4f1a-21bc-11ea-a13a-137349068a90,3,3,2013-02-05 10:12:39.000,125,1024,1280,empty
a3609808-21bc-11ea-a13a-137349068a90,-1,92edf54c-21bc-11ea-a13a-137349068a90.jpg,0,960b7f9c-21bc-11ea-a13a-137349068a90,1,3,2013-02-06 14:04:38.000,125,1024,1280,empty
9ee111a4-21bc-11ea-a13a-137349068a90,-1,885433c6-21bc-11ea-a13a-137349068a90.jpg,0,960b7f9c-21bc-11ea-a13a-137349068a90,2,3,2013-02-06 14:04:39.000,125,1024,1280,empty
...,...,...,...,...,...,...,...,...,...,...,...
9ae2935c-21bc-11ea-a13a-137349068a90,1,8e215ad6-21bc-11ea-a13a-137349068a90.jpg,0,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-07-28 21:19:03.000,218,1024,1280,empty
9b0103d2-21bc-11ea-a13a-137349068a90,1,956ebda6-21bc-11ea-a13a-137349068a90.jpg,0,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-07-28 21:19:03.000,218,1024,1280,empty
a154e0dc-21bc-11ea-a13a-137349068a90,1,8d6dc49e-21bc-11ea-a13a-137349068a90.jpg,0,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-07-28 21:19:04.000,218,1024,1280,empty


### Agora será feita a junção das pastas empty e no_empty em uma unica pasta

In [873]:
pasta_empty_no_empty = [pasta_empty, pasta_no_empty]

In [874]:
pasta_empty_no_empty = pd.concat(pasta_empty_no_empty)

In [875]:
pasta_empty_no_empty

,count,image_id,category_id,seq_id,frame_num,seq_num_frames,datetime,location,height,width,name
id,,,,,,,,,,,
9ab32f72-21bc-11ea-a13a-137349068a90,-1,888e07e0-21bc-11ea-a13a-137349068a90.jpg,0,8ede4f1a-21bc-11ea-a13a-137349068a90,1,3,2013-02-05 10:12:37.000,125,1024,1280,empty
9aafd386-21bc-11ea-a13a-137349068a90,-1,8ede4d44-21bc-11ea-a13a-137349068a90.jpg,0,8ede4f1a-21bc-11ea-a13a-137349068a90,2,3,2013-02-05 10:12:38.000,125,1024,1280,empty
9d80ea64-21bc-11ea-a13a-137349068a90,-1,868b2ab8-21bc-11ea-a13a-137349068a90.jpg,0,8ede4f1a-21bc-11ea-a13a-137349068a90,3,3,2013-02-05 10:12:39.000,125,1024,1280,empty
a3609808-21bc-11ea-a13a-137349068a90,-1,92edf54c-21bc-11ea-a13a-137349068a90.jpg,0,960b7f9c-21bc-11ea-a13a-137349068a90,1,3,2013-02-06 14:04:38.000,125,1024,1280,empty
9ee111a4-21bc-11ea-a13a-137349068a90,-1,885433c6-21bc-11ea-a13a-137349068a90.jpg,0,960b7f9c-21bc-11ea-a13a-137349068a90,2,3,2013-02-06 14:04:39.000,125,1024,1280,empty
...,...,...,...,...,...,...,...,...,...,...,...
9b4c9de2-21bc-11ea-a13a-137349068a90,1,8b7899a2-21bc-11ea-a13a-137349068a90.jpg,372,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-08-01 11:39:03.000,218,1024,1280,meleagris ocellata
9a379740-21bc-11ea-a13a-137349068a90,1,958f609c-21bc-11ea-a13a-137349068a90.jpg,372,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-08-01 11:39:04.000,218,1024,1280,meleagris ocellata
9d0a3c0c-21bc-11ea-a13a-137349068a90,1,93bd2ede-21bc-11ea-a13a-137349068a90.jpg,372,995a6da2-21bc-11ea-a13a-137349068a90,-1,-1,2013-08-01 11:39:05.000,218,1024,1280,meleagris ocellata
